In [9]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
from scipy import special
from IPython.display import display
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
py.offline.init_notebook_mode(connected = True)

In [6]:
kinds = ['type', 'area', 'jour', 'nat', 'org']
Name_kinds = ['Publication type', 'Research field', 'Publication source', 'Nation', 'Organization']

all_frame = []
for i, kind in enumerate (kinds):
    kind_data = pd.DataFrame()
    for year in range(1996, 2020):
        temp = pd.read_csv(f'OLED_data/{year}_{kind}.txt', header = 0, delimiter = '\t', index_col = [0], usecols = [0,1], names = [f'{Name_kinds[i]}', f'{year}'])
        temp.index = [temp.index[x].title() for x in range(len(temp.index))]
        temp.drop(temp.tail(1).index, inplace=True)
        kind_data = pd.concat([kind_data, temp], axis=1, sort = True)
    all_frame.append(kind_data)
    
alldata = pd.concat(all_frame, keys=Name_kinds, axis=0)   

In [10]:
def plot_bar(plot, kind, year, mode):
    if mode == 'All time':
        temp = alldata.loc[f'{kind}'].fillna(0).sum(axis=1)
        temp = temp.sort_values(ascending=False)
        temp = temp.head(15)
    elif mode == 'Last 5 years':
        temp = alldata.loc[f'{kind}'][['2015', '2016', '2017', '2018', '2019']].sum(axis=1)
        temp = temp.sort_values(ascending=False)
        temp = temp.head(15)
    else:    
        temp = alldata.loc[f'{kind}'][f'{year}'].sort_values(ascending=False)
        temp = temp.head(15)
    if plot == 'Bar chart':
        fig = go.Figure(go.Bar(
                    x= temp,
                    y= temp.index,
                    text=temp ,
                    marker={'color': temp, 
                            'colorscale': 'BuRg'},
                    textposition='auto',
                    orientation='h',))

    else:
        fig = px.treemap(path=[temp.index], values=temp, color=temp, color_continuous_scale='RdBu')
        
    fig.update_yaxes(automargin=True)
    fig.update_layout(margin=dict(l=20, r=20, t=30, b=20), title_text=f'Top 15 paper {kind}',template='plotly_dark')
    fig.show()


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border=None,
                    width='100%')
    
radio1 = widgets.RadioButtons(
    options=['Select a year', 'Last 5 years', 'All time'],
   value='All time',
   layout={'width': 'max-content'},
    description='Mode',
    disabled=False
)

radio2 = widgets.RadioButtons(
    options=['Bar chart', 'Tree map'],
   value='Bar chart',
   layout={'width': 'max-content'},
    description='Plot',
    disabled=False
)

index = widgets.Dropdown(
    options    = Name_kinds ,
    value      = Name_kinds[0],
    description='Index:',
    disabled   =False,
)

Select_year = widgets.IntSlider(
    min  = 1996, 
    max  = 2019, 
    value= 2019, 
    step = 1,
    disabled=True
)

def help(k):
    if k != 'Select a year':
        Select_year.disabled = True
    else:
        Select_year.disabled = False

widgets.interactive(help , k = radio1)

ui = widgets.HBox([radio2, index, radio1, Select_year], layout=box_layout)
out = widgets.interactive_output(plot_bar, {'plot': radio2, 'year': Select_year, 'kind': index, 'mode': radio1})
display(ui, out) 



Output()

In [19]:
nation_code = pd.read_csv('country_code.csv', header = None, delimiter = ',', skipfooter =0)
nation_code.columns = ['Nation', 'Nation_code']
nationRank = alldata.loc['Nation'].fillna(0).sum(axis=1)

fig = px.choropleth(nationRank, locations=nation_code['Nation_code'], color= nationRank,
                            hover_name = nationRank.index,
                           labels={'Amount':'Amount'},
                    color_continuous_scale='Reds',
                    title = 'Total publication regarding OLEDs',
                    projection = 'orthographic',
                    range_color = [0, 4000],
                    color_continuous_midpoint = 10

                          )
fig.update_layout(margin={"r":10,"t":30,"l":10,"b":10})
fig.show()